In [1]:
from finetune_utils import create_mistral_finetune_data
import json
import together

In [11]:
%env TOGETHER_API_KEY=eb25e0d054aec138257edf3c264089a5fa04be265ce8c3890597884cec1bc622

env: TOGETHER_API_KEY=eb25e0d054aec138257edf3c264089a5fa04be265ce8c3890597884cec1bc622


## ONLY RUN THESE DATA UPLOAD CELLS ONCE!

In [3]:
with open('data/splits/connections_train.json', 'r') as file:
    data = json.load(file)

In [4]:
train_data = create_mistral_finetune_data(data)
print(train_data)

['<s>[INST] Given 16 items, find groups of four items that share something in common.\n\nCategory Examples\nFish: BASS, FLOUNDER, SALMON, TROUT\nFire ___: ANT, DRILL, ISLAND, OPAL\nCategories will always be more specific than “5-letter-words,” “Names” or “Verbs.”\nEach puzzle has exactly one solution. Watch out for words that seem to belong to multiple categories!\n\nNow answer for these 16 words. Follow these restrictions for the output:\n1. Respond in 4 lines, with a group of 4 words on each line. ONLY include the 16 words given. There should be NO OTHER WORDS.\n2. DO NOT include the descriptions. DO NOT have any "Descriptions:" text.\n3. DO NOT include any preceding text, like "Answers:", or line numbers, like "1.".\n\nNECK, MOON, HORSESHOE, AGAIN, LEAD, NICKEL, ZINC, CLOVER, RAINBOW, HALF, QUARTER, SO, TIN, DIME, IRON, PENNY [/INST]\nDIME, NICKEL, PENNY, QUARTER\nIRON, LEAD, TIN, ZINC\nCLOVER, HORSESHOE, MOON, RAINBOW\nAGAIN, HALF, NECK, SO </s>', '<s>[INST] Given 16 items, find gr

In [12]:
!together files upload data/splits/connections_train.jsonl

Uploading file connections_train.jsonl: 100%|█| 280k/280k [00:01<00:00, 163kB/s]
{
    "id": "file-312c091b-47a8-4387-a15d-7eed376b43f7",
    "object": "file",
    "created_at": 1716491840,
    "type": null,
    "purpose": "fine-tune",
    "filename": "connections_train.jsonl",
    "bytes": 0,
    "line_count": 0,
    "processed": false,
    "FileType": "jsonl"
}


In [9]:
%env WANDB=191ee4748b3456523ee47c1c1ec1e3f476ade698

env: WANDB=191ee4748b3456523ee47c1c1ec1e3f476ade698


In [ ]:
ft_resp = together.Finetune.create(
  training_file = "file-312c091b-47a8-4387-a15d-7eed376b43f7" ,
  model = "mistralai/Mistral-7B-Instruct-v0.2",
  n_epochs = 8,
  batch_size = 1,
  n_checkpoints = 8,
  learning_rate = 5e-5,
  wandb_api_key = "191ee4748b3456523ee47c1c1ec1e3f476ade698",
  estimate_price = True,
)


In [10]:
!together fine-tuning create --training-file "file-c57cf573-06ef-4212-a707-973da493d778" --model "mistralai/Mistral-7B-Instruct-v0.2" --wandb-api-key $WANDB

{
    "id": "ft-d3e16b21-0665-403c-bd8b-a42691fc082f",
    "training_file": "file-c57cf573-06ef-4212-a707-973da493d778",
    "validation_file": "",
    "model": "mistralai/Mistral-7B-Instruct-v0.2",
    "output_name": "emily49@stanford.edu/Mistral-7B-Instruct-v0.2-2024-05-23-18-59-23",
    "n_epochs": 1,
    "n_checkpoints": 1,
    "batch_size": 16,
    "learning_rate": 3e-05,
    "eval_steps": 0,
    "lora": false,
    "lora_r": 8,
    "lora_alpha": 8,
    "lora_dropout": 0,
    "created_at": "2024-05-23T18:59:23.645Z",
    "updated_at": "2024-05-23T18:59:23.645Z",
    "status": "pending",
    "job_id": "",
    "events": [
        {
            "object": "fine-tune-event",
            "created_at": "2024-05-23T18:59:23.645Z",
            "level": "",
            "message": "Fine tune request created",
            "type": "JOB_PENDING",
            "param_count": 0,
            "token_count": 0,
            "wandb_url": "",
            "hash": "",
            "total_steps": 0,
        